### Test the prediction accuracy on the mechanical Turk dataset

### Imports

In [1]:
import speech_recognition as sr 
import pyttsx3  
import pyaudio
import os
import numpy as np

### Declarations

In [2]:
main_path = '/Users/sahityasridhar/Documents/OS/keyword/dataset/mturk_ds'

In [3]:
WORDS = ['choose','duet','like','save','share','follow','request','add','join']

### Soundex algorithm for phonetic similarity

In [4]:
def soundex(query):
    query = query.lower()
    letters = [char for char in query if char.isalpha()]
    if len(query) == 1:
        return query + "000"
    to_remove = ('a', 'e', 'i', 'o', 'u', 'y', 'h', 'w')
    first_letter = letters[0]
    letters = letters[1:]
    letters = [char for char in letters if char not in to_remove]
    if len(letters) == 0:
        return first_letter + "000"
    to_replace = {('b', 'f', 'p', 'v'): 1, ('c', 'g', 'j', 'k', 'q', 's', 'x', 'z'): 2,
                  ('d', 't'): 3, ('l',): 4, ('m', 'n'): 5, ('r',): 6}
    first_letter = [value if first_letter else first_letter for group, value in to_replace.items()
                    if first_letter in group]
    letters = [value if char else char
               for char in letters
               for group, value in to_replace.items()
               if char in group]

    letters = [char for ind, char in enumerate(letters)
               if (ind == len(letters) - 1 or (ind+1 < len(letters) and char != letters[ind+1]))]

    if first_letter == letters[0]:
        letters[0] = query[0]
    else:
        letters.insert(0, query[0])

    first_letter = letters[0]
    letters = letters[1:]
    letters = [char for char in letters if isinstance(char, int)][0:3]
    while len(letters) < 3:
        letters.append(0)
    letters.insert(0, first_letter)
    string = "".join([str(l) for l in letters])
    return string

In [5]:
words_soundex = {}
for word in WORDS:
    words_soundex[word] = soundex(word)

### Edit distance for textual similarity

In [6]:
def edit_distance(string,word):
    m,n = len(string),len(word)
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)] 
    for i in range(m + 1): 
        for j in range(n + 1): 
            if i == 0: 
                dp[i][j] = j    # Min. operations = j 
            elif j == 0: 
                dp[i][j] = i    # Min. operations = i 
            elif string[i-1] == word[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
            else: 
                dp[i][j] = 1 + min(dp[i][j-1],dp[i-1][j],dp[i-1][j-1])    # Replace 
    return dp[m][n]

### Similarity computation - driver function

In [7]:
def similarity(string,words_soundex):
    s = soundex(string)
    edits = np.array([edit_distance(string,word) for word in words_soundex])
    soundex_edits = np.array([edit_distance(words_soundex[word],soundex(string)) for word in words_soundex])
    soundex_phonetics = np.array([0.5*(s[1:]==words_soundex[word][1:]) + 0.5*(s[0]==words_soundex[word][0]) for word in words_soundex])
    weighted = 0.4*edits + 0.4*soundex_edits + 0.2*soundex_phonetics
    target = WORDS[np.argmin(weighted)]
    return target

### Speech - text on Mechanical Turk data

In [8]:
def read_file_and_predict(file,label):
    r = sr.Recognizer()
    with sr.AudioFile(file) as source: 
        audio = r.record(source) 
        try:
            a = r.recognize_google(audio,language='en-US')
        except:
            return -1
        a = similarity(a,words_soundex)
        if a==label:
            return 0
        return 1

In [9]:
def traverse_folders(rootdir):
    wrong,total,unpredicted = 0,0,0
    for i, (subdirs, dirs, files) in enumerate(os.walk(rootdir)):
        for file in files:
            if not file.startswith('.'):
                fname,_ = file.split('.')
                folder = subdirs.split('/')[-1]
                status = read_file_and_predict(os.path.join(subdirs,file),folder)
                wrong+=status if status!=-1 else 0
                total+=1 if status!=-1 else 0
                unpredicted+=1 if status==-1 else 0
    return wrong,total,unpredicted

### Performance Analysis

In [10]:
wrong,total,unpredicted = traverse_folders(main_path)

In [11]:
print('Total number of samples : ',total+unpredicted)
print('No match : ',unpredicted,' ',int((unpredicted/total)*100),'%')
print('Correct predictions : ',total-wrong,' ',int(((total-wrong)/total) * 100),'%')

Total number of samples :  909
No match :  72   8 %
Correct predictions :  785   93 %
